In [8]:
import string
# Библиотека построения индекса приближенного поиска ближайших соседей
import annoy
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import FastText
from tqdm.auto import tqdm

In [2]:
# Для фильтрации пунктуации
exclude = set(string.punctuation)
# Для приведения слов в начальной форме
morpher = MorphAnalyzer()

# Для фильтрации стоп-слов
sw = get_stop_words("ru")

def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [4]:
# Загрузим текстовые описания товаров и преобразуем к нормализованному виду
shop_data = pd.read_csv("ProductsDataset.zip")
shop_data['text'] = shop_data['title'] + " " + shop_data["descrirption"]
shop_data['text'] = shop_data['text'].apply(lambda x: preprocess_txt(str(x)))

# Обучим модель fasttext на текстах
sentences = [i for i in shop_data['text'] if len(i) > 2]

In [6]:
modelFT = FastText(sentences=sentences, vector_size=20, min_count=1, window=5)

In [16]:
# Для того, чтобы быстро находить айтемы положим эмбединги их тайтлов в ANN индекс
# Создадим объект индекса
ft_index_shop = annoy.AnnoyIndex(20 ,'angular')

index_map_shop = {}
counter = 0

for i in tqdm(range(len(shop_data))):
    n_ft = 0
    index_map_shop[counter] = (shop_data.loc[i, "title"], shop_data.loc[i, "image_links"])
    vector_ft = np.zeros(20)
    # Каждое слово обернем в эмбеддинг
    for word in shop_data.loc[i, "text"]:
        if word in modelFT.wv.key_to_index.keys():
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    ft_index_shop.add_item(counter, vector_ft)
    counter += 1

# 
ft_index_shop.build(10)

  0%|          | 0/35548 [00:00<?, ?it/s]

True

In [17]:
shop_data.head()

,title,descrirption,product_id,category_id,subcategory_id,properties,image_links,text
0,Юбка детская ORBY,"Новая, не носили ни разу. В реале красивей чем...",58e3cfe6132ca50e053f5f82,22.0,2211,"{'detskie_razmer_rost': '81-86 (1,5 года)'}",http://cache3.youla.io/files/images/360_360/58...,"[юбка, детский, orby, новый, носить, реал, кра..."
1,Ботильоны,"Новые,привезены из Чехии ,указан размер 40,но ...",5667531b2b7f8d127d838c34,9.0,902,"{'zhenskaya_odezhda_tzvet': 'Зеленый', 'visota...",http://cache3.youla.io/files/images/360_360/5b...,"[ботильон, новыепривезти, чехия, указать, разм..."
2,Брюки,Размер 40-42. Брюки почти новые - не знаю как ...,59534826aaab284cba337e06,9.0,906,{'zhenskaya_odezhda_dzhinsy_bryuki_tip': 'Брюк...,http://cache3.youla.io/files/images/360_360/59...,"[брюки, размер, 4042, брюки, новый, знать, мер..."
3,Продам детские шапки,"Продам шапки,кажда 200р.Розовая и белая проданны.",57de544096ad842e26de8027,22.0,2217,"{'detskie_pol': 'Девочкам', 'detskaya_odezhda_...",http://cache3.youla.io/files/images/360_360/57...,"[продать, детский, шапка, продать, шапкикажда,..."
4,Блузка,"Темно-синяя, 42 размер,состояние отличное,как ...",5ad4d2626c86cb168d212022,9.0,907,"{'zhenskaya_odezhda_tzvet': 'Синий', 'zhenskay...",http://cache3.youla.io/files/images/360_360/5a...,"[блузка, темносиний, 42, размерсостояние, отли..."


In [24]:
# Попробуем порекомендовать что-то
# Так как у нас нет пользовательских сессий, то оценка качества "На глаз"

def recommend(item_id):
    # Получим тайтл айтема по идентификатору
    title = shop_data[shop_data['product_id']==item_id]['title'].values[0]
    title = preprocess_txt(title)
    
    i = shop_data[shop_data['product_id']==item_id].index[0]
    
    vector_ft = np.zeros(20)
    n_ft = 0
    # Каждое слово обернем в эмбеддинг
    for word in shop_data.loc[i, "text"]:
        if word in modelFT.wv.key_to_index.keys():
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    # Найдем ближайшие по тайтлу
    rec_items = ft_index_shop.get_nns_by_vector(vector_ft, 5)
    return [index_map_shop[i][0] for i in rec_items]

In [25]:
recommend("5667531b2b7f8d127d838c34")

['Ботильоны',
 'Туфли',
 'Сабо',
 'Босоножки женские размер 40 маломерки',
 'Босоножки']

In [26]:
recommend("5ad4d2626c86cb168d212022")

['Блузка', 'Болеро', 'Блузка', 'Новая Женская блузка', 'Блузка Tally Weijl']

In [27]:
recommend("57de544096ad842e26de8027")

['Продам детские шапки',
 'Шапка скомороха',
 'ШАПКА ДЕТСКАЯ С 5-8 ЛЕТ.РАЗМ.52',
 'Шарф Accessorize',
 'Шапка новая']